#Q 1

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
with open("dataset path") as file:
    data = file.readlines()

In [ ]:
len(data)

In [ ]:
for i in range(len(data)):
    data[i] = data[i][:-1]

In [ ]:
data_dict = dict()

for i in range(len(data)):
    split_data = data[i].split()
    data_dict[split_data[0]] = np.array(split_data[1:]).astype('float64')

In [ ]:
data_dict["cat"]

In [ ]:
def cosine_similarity(a, b):
    nominator = np.dot(a, b)
    
    a_norm = np.sqrt(np.sum(a**2))
    b_norm = np.sqrt(np.sum(b**2))
    
    denominator = a_norm * b_norm
    
    cosine_similarity = nominator / denominator
    
    return cosine_similarity

In [ ]:
def find_word(a, b, c, data_dict):
    a, b, c = a.lower(), b.lower(), c.lower()
    a_vector, b_vector, c_vector = data_dict[a], data_dict[b], data_dict[c]
    
    all_words = data_dict.keys()
    max_cosine_similarity = -1000
    best_match_word = None
    
    for word in all_words:
        if word in [a, b, c]:
            continue
            
        cos_sim = cosine_similarity(np.subtract(b_vector, a_vector), np.subtract(data_dict[word], c_vector))
        
        if cos_sim > max_cosine_similarity:
            max_cosine_similarity = cos_sim
            best_match_word = word
            
    return best_match_word, cos_sim

In [ ]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32,
                                          return_sequences=True,
                                          input_shape=(window_size, 1)))
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(72))

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')words_bag = [
    ('boy', 'girl', 'man'),
    ('black', 'white', 'blue'),
    ('book', 'library', 'coffee'),
    ('orange', 'banana', 'apple'),
    ('Canada', 'America', 'Ottawa')
]

for words in words_bag:
    d, cos_sim = find_word(*words, data_dict)
    print("({}, {}) ----> ({}, {}) with {} similarity score".format(*words, d, cos_sim))

(boy, girl) ----> (man, woman) with 0.07664205394715363 similarity score
(black, white) ----> (blue, mccurry) with -0.00515067279270105 similarity score
(book, library) ----> (coffee, libraries) with 0.06959056667990891 similarity score
(orange, banana) ----> (apple, bananas) with 0.01433175480862464 similarity score
(Canada, America) ----> (Ottawa, latin) with 0.3122692977134148 similarity score


#Q2

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("dataset path")
data

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 19:00:00.000 +0200,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
96449,2016-09-09 20:00:00.000 +0200,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
96450,2016-09-09 21:00:00.000 +0200,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
96451,2016-09-09 22:00:00.000 +0200,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


In [ ]:
data.info()

In [ ]:
data.plot(subplots=True, figsize=(12, 12))
plt.xticks(rotation=45)
plt.show()

In [ ]:
uni_data = data['Temperature (C)']
uni_data.index = data['Formatted Date']
uni_data.head()

In [ ]:
uni_data.plot(subplots=True, figsize=(10, 6))
plt.xticks(rotation=45)
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#Reshape the data
uni_data = np.array(uni_data).reshape(-1, 1)

#Normalize the data
scaler = MinMaxScaler()
uni_data = scaler.fit_transform(uni_data)

In [ ]:
def prepare_data(data, window_size, horizon):
    X, y = [], []
    for i in range(len(data) - window_size - horizon):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon])
    return np.array(X), np.array(y)

In [ ]:
window_size = 30
horizon = 7

X, y = prepare_data(uni_data, window_size, horizon)

In [ ]:
from sklearn.model_selection import train_test_split

val_split = 0.2

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_split, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense ,Dropout, BatchNormalization

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(window_size, 1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(7))


model.compile(loss='mean_squared_error', optimizer='adam')

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

In [ ]:
model.fit(X_train, y_train, epochs= 60, batch_size=64,  validation_split= val_split, callbacks=[early_stopping])

Epoch 1/60
965/965 [==============================] - 14s 9ms/step - loss: 0.1524 - val_loss: 0.0172
Epoch 2/60
965/965 [==============================] - 9s 9ms/step - loss: 0.0092 - val_loss: 0.0053
Epoch 3/60
965/965 [==============================] - 9s 9ms/step - loss: 0.0050 - val_loss: 0.0037
Epoch 4/60
965/965 [==============================] - 8s 8ms/step - loss: 0.0042 - val_loss: 0.0038
Epoch 5/60
965/965 [==============================] - 9s 9ms/step - loss: 0.0041 - val_loss: 0.0034
Epoch 6/60
965/965 [==============================] - 9s 10ms/step - loss: 0.0040 - val_loss: 0.0038
Epoch 7/60
965/965 [==============================] - 9s 9ms/step - loss: 0.0039 - val_loss: 0.0040
Epoch 8/60
965/965 [==============================] - 9s 10ms/step - loss: 0.0038 - val_loss: 0.0033
Epoch 9/60
965/965 [==============================] - 9s 9ms/step - loss: 0.0037 - val_loss: 0.0040
Epoch 10/60
965/965 [==============================] - 8s 8ms/step - loss: 0.0035 - val_loss: 0.0

In [ ]:
Evaluate = model.evaluate(X_val, y_val)
print('Evaluation :', Evaluate)

603/603 [==============================] - 2s 4ms/step - loss: 0.0027
Evaluation : 0.0027404511347413063


In [ ]:
last_30_days = uni_data[-window_size:]

predicted_temperatures = []

for _ in range(7):
    X_pred = np.array([last_30_days])
    X_pred = np.reshape(X_pred, (X_pred.shape[0], X_pred.shape[1], 1))
    predicted_temperature = model.predict(X_pred)
    predicted_temperatures.append(predicted_temperature[0][0])
    last_30_days = np.append(last_30_days[1:], predicted_temperature)

predicted_temperatures = np.array(predicted_temperatures)
predicted_temperatures = predicted_temperatures.reshape(-1, 1)
predicted_temperatures = scaler.inverse_transform(predicted_temperatures)

print('Predicted Temperatures:', predicted_temperatures)

1/1 [==============================] - 0s 19ms/step
Predicted Temperatures: [[15.326774]
 [19.319525]
 [15.638637]
 [14.398827]
 [15.0094  ]
 [13.283896]
 [13.723785]]
